In [ ]:
import sqlite3
import pandas as pd

# 1. BAĞLANTIYI OLUŞTUR
db_name = "ticaret_db.db"
connection = sqlite3.connect(db_name)
cursor = connection.cursor()

# Foreign Key desteğini aktif et (SQLite için zorunludur)
cursor.execute("PRAGMA foreign_keys = ON;")

# ---------------------------------------------------------
# 2. TABLOLARI OLUŞTURMA (CREATE)
# ---------------------------------------------------------
# Hata almamak için önce çocukları (bağımlıları), sonra babaları siliyoruz
tablolar = ["siparis", "urun", "musteri", "firma"]
for tablo in tablolar:
    cursor.execute(f"DROP TABLE IF EXISTS {tablo}")

# A. Firma Tablosu (Bağımsız)
cursor.execute("""
CREATE TABLE firma (
    fId INTEGER PRIMARY KEY,
    firmaAdi TEXT,
    sehir TEXT
);
""")

# B. Müşteri Tablosu (Bağımsız)
cursor.execute("""
CREATE TABLE musteri (
    mId INTEGER PRIMARY KEY,
    musteriadi TEXT,
    sehir TEXT
);
""")

# C. Ürün Tablosu (Firmaya Bağlı - fId ile)
cursor.execute("""
CREATE TABLE urun (
    uId INTEGER PRIMARY KEY,
    urunadi TEXT,
    fId INTEGER,
    FOREIGN KEY(fId) REFERENCES firma(fId)
);
""")

# D. Sipariş Tablosu (Ürüne ve Müşteriye Bağlı - uId ve mId ile)
cursor.execute("""
CREATE TABLE siparis (
    sipId INTEGER PRIMARY KEY,
    uId INTEGER,
    mId INTEGER,
    miktar INTEGER,
    fiyat REAL,
    FOREIGN KEY(uId) REFERENCES urun(uId),
    FOREIGN KEY(mId) REFERENCES musteri(mId)
);
""")

print("Tablolar başarıyla oluşturuldu.")

# ---------------------------------------------------------
# 3. VERİ EKLEME (INSERT)
# ---------------------------------------------------------

# Firmalar
cursor.executemany("INSERT INTO firma VALUES (?,?,?)", [
    (10, 'TeknoSA', 'İstanbul'),
    (20, 'Vatan Bilgisayar', 'Ankara'),
    (30, 'Defacto', 'İzmir')
])

# Müşteriler
cursor.executemany("INSERT INTO musteri VALUES (?,?,?)", [
    (100, 'Ahmet Yılmaz', 'Bursa'),
    (101, 'Ayşe Demir', 'İstanbul'),
    (102, 'Mehmet Öz', 'Ankara')
])

# Ürünler (Hangi firma üretiyor/satıyor?)
cursor.executemany("INSERT INTO urun VALUES (?,?,?)", [
    (1, 'Laptop', 10),      # TeknoSA satıyor
    (2, 'Mouse', 10),       # TeknoSA satıyor
    (3, 'Klavye', 20),      # Vatan satıyor
    (4, 'Tişört', 30)       # Defacto satıyor
])

# Siparişler (Kim, hangi ürünü, kaç tane, kaça aldı?)
cursor.executemany("INSERT INTO siparis VALUES (?,?,?,?,?)", [
    (5001, 1, 100, 1, 15000.0),  # Ahmet, 1 tane Laptop aldı
    (5002, 3, 101, 2, 500.0),    # Ayşe, 2 tane Klavye aldı
    (5003, 4, 100, 3, 100.0),    # Ahmet, 3 tane Tişört aldı
    (5004, 1, 102, 1, 16000.0)   # Mehmet, 1 tane Laptop aldı
])

connection.commit()
print("Veriler eklendi.")

# ---------------------------------------------------------
# 4. RAPORLAMA (PANDAS İLE KONTROL)
# ---------------------------------------------------------
# Tüm tabloları birleştiren (JOIN) örnek bir rapor
sql_sorgu = """
SELECT 
    s.sipId,
    m.musteriadi,
    u.urunadi,
    f.firmaAdi,
    s.miktar,
    s.fiyat,
    (s.miktar * s.fiyat) as ToplamTutar
FROM siparis s
JOIN musteri m ON s.mId = m.mId
JOIN urun u ON s.uId = u.uId
JOIN firma f ON u.fId = f.fId;
"""

print("-" * 50)
print("SİPARİŞ DETAY RAPORU:")
df = pd.read_sql(sql_sorgu, connection)
print(df)

def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None

In [ ]:
#isa yakar isimli müşterinin satın aldığı diş macunu ürünün miktarını 10,fiyatını 500 olarak güncelleyen sorgu

In [ ]:
#siparis tablosuna siparisTarihi adında bir sütun ekleyiniz girilecek tarihin 2020-2050 arasında olmasını saglayınız

In [ ]:
#ürün tablosuna 3 adet ürün ekleyen kod

In [ ]:
#colgate firmasına ait tüm ürünleri sipariş tablosundan silen kod

In [ ]:
#tüm müşterinin adını, lehir bilgisini, hangi ürünleri , hangi firmadan kaç adet, ne kadara aldıklarını gösteren kod

In [1]:
#sipariş veren müşterilerin adını ilini ,kaçar adet siparis verdiklerini büyükten küçüğe doğru sıralayan kod

In [ ]:
#beşten fazla ürüne sahip olan firmanın adını ve şehir bilgilerini gösteren

In [ ]:
#sadece en fazla siparis veren müşterinin adını ilini toplam alışveriş fiyatını gösterin

In [ ]:
#aynı ilde bulunan müşterileri ve firmaları listeleyen

In [ ]:
connection.close()